# Tensorflow tutorial: Quickstart for beginners

In [1]:
import tensorflow as tf

2023-08-01 19:36:59.521175: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-01 19:36:59.568571: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-01 19:36:59.569370: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-01 19:37:00.484336: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Load & prepare MNIST

In [2]:
mnist = tf.keras.datasets.mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

# normalize data
X_train, X_test = X_train / 255.0, X_test / 255.0

11490434/11490434 [==============================] - 0s 0us/step


Build a sequential neural net

In [3]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)), 
    tf.keras.layers.Dense(128, activation='relu'), 
    tf.keras.layers.Dropout(0.2), 
    tf.keras.layers.Dense(10)
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101770 (397.54 KB)
Trainable params: 101770 (397.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


For each example, the model returns logits, or log-odds scores for each class

**logits**: non-normalized predictions (usually input to softmax which normalizes the probabilities)<br> 

**log-odds**: logarithm of the odds of an event: $\text{log-odds} = \ln(\frac{p}{1-p})$. This is the inverse of sigmoid.

In [4]:
predictions = model(X_train[:1]).numpy()
predictions

array([[-0.12894067, -0.3003755 ,  0.1621922 , -0.84611845, -0.05044453,
         0.24006858, -0.0144949 , -0.17953241,  0.00139112,  0.17785925]],
      dtype=float32)

Now use `tf.nn.softmax` to convert to probabilities

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.09287914, 0.07824651, 0.12426694, 0.0453369 , 0.10046358,
        0.1343312 , 0.10414091, 0.08829711, 0.1058085 , 0.12622917]],
      dtype=float32)

**Note**: you can put tf.nn.softmax as the last layer. Discouraged b/c softmax output makes it impossible to provide an exact / stable loss calculation. 

`losses.SparseCategoricalCrossentropy` - take a vector of logits & returns a scalar loss

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This is the negative log probability of the true class: zero if model is 100% sure that this is the correct class. 

Since the model is untrained, the probabilities will be close to random, so loss should be log(1/10) ~ 2.3

In [7]:
loss_fn(y_train[:1], predictions).numpy

<bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(), dtype=float32, numpy=2.0074468>>

Now compile and train the model

In [8]:
model.compile(optimizer='adam', 
             loss=loss_fn, 
             metrics=['accuracy'])

In [9]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.3008 - accuracy: 0.9133
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1431 - accuracy: 0.9570
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1058 - accuracy: 0.9680
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0849 - accuracy: 0.9734
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0727 - accuracy: 0.9766


`Model.evaluate` - check performance on a validation or test set

In [10]:
model.evaluate(X_test, y_test, verbose=2)

313/313 - 1s - loss: 0.0705 - accuracy: 0.9789 - 598ms/epoch - 2ms/step


[0.07050502300262451, 0.9789000153541565]